In [105]:
from tensorboard.backend.event_processing import event_accumulator
import pandas as pd
from tbparse import SummaryReader


In [106]:
SAVE_PATH = f"../results/scanEZ_results.md"
OVERWRITE=True
if OVERWRITE:
     open(SAVE_PATH, 'w').close()


In [107]:
def get_metric_results_from_eventfile(dirpath, metrics, SAVE_PATH="", title="", select="latest"):
    # select = "latest", or "lowest" or "highest"
    reader = SummaryReader(dirpath, pivot=False, extra_columns={'dir_name','wall_time'})
    df = reader.scalars
    df = df.rename(columns={"tag": "metric", "dir_name": "fold"})
    grouped = df.groupby(["fold","metric"])
    # ['step'].max().reset_index() # this is choosing the max AUROC but we want the latest!!
    if select=='latest':
        df = df.loc[grouped['wall_time'].idxmax()].reset_index(drop=True)
    elif select=='highest':
        df = df.loc[grouped['value'].idxmax()].reset_index(drop=True)
    elif select=='lowest':
        df = df.loc[grouped['value'].idxmin()].reset_index(drop=True)
    
    df = df[df["metric"].isin(metrics)].set_index(["fold"]).sort_values("metric").reset_index()

    res = df.pivot(index='fold', columns='metric', values='value')#.rename_axis(columns=None)
    if SAVE_PATH:
        with open(SAVE_PATH,'a') as f:
            f.write("\n\n")
            f.write(title)
            f.write("\n")
        res.to_markdown(SAVE_PATH, mode="a", tablefmt="github", index=True,floatfmt='.3f')
    return res

In [108]:
metrics = ["train_auroc", "val_auroc", "train_accuracy_epoch", "val_accuracy_epoch"]
pooling_methods = ["mean", "final_pos", "masked_mean"]
versions = ["ptEZftEML","ptEZ", "ptEML"]
labels = ['SVT','Rote_X', 'Rote_Y', "Rote_Z",
          'Inference_X', "Inference_Y","Inference_Z",
          "Deep_X", "Deep_Z",
          "Rote_D","Inference_D",
          "MW"]
# labels=["Rote_X"]
classifier_names = [f"{version}_{pooling}" for version in versions for pooling in pooling_methods]
mean_aurocs=[]
for classifier in classifier_names:
    val_aurocs = []
    for label in labels:
        res=get_metric_results_from_eventfile(f"../lightning_logs/2025/classifiers/scanez/{label}_scanez_{classifier}", metrics, SAVE_PATH=None, select="latest")
        res['label']=label
        val_aurocs.append({"label":label, 
                        "mean AUROC": res[['val_auroc']].mean().item(),
                        "min AUROC": res[['val_auroc']].min().item(),
                        "max AUROC": res[['val_auroc']].max().item()
                        })
        mean_aurocs.append({"label":label, "classifier":classifier, "mean AUROC": res[['val_auroc']].mean().item()})
    tit=f"Held-out AUROCs: {classifier}"
    with open(SAVE_PATH, "a") as f:
        f.write("\n\n")
        f.write(tit)
        f.write("\n")
    tb = pd.DataFrame.from_records(val_aurocs)
    tb.to_markdown(SAVE_PATH, mode="a", tablefmt="github", index=False,floatfmt='.3f')


In [109]:

tb_all = pd.DataFrame.from_records(mean_aurocs)
tb_all = tb_all.pivot(index='label', columns='classifier', values='mean AUROC')
# # transpose to have label in  columns and one row per classifier
tb_all = tb_all.T
# make bold the highest val per row
def highlight_max(s):
    is_max = s == s.max()
    return ['**{:.3f}**'.format(v) if is_max else '{:.3f}'.format(v) for v, is_max in zip(s, is_max)]

tb_all_fmt = tb_all.apply(highlight_max, axis=0)
tb_all_fmt = tb_all_fmt.T

# make it have 2 row header - split current header on first underscore
# tb_all.columns = pd.MultiIndex.from_tuples([col.split('_', 1) for col in tb_all.columns])
# Display the column headers split across 2 rows
multiheader = pd.MultiIndex.from_tuples([(col.split('_')[0], col.split('_')[1]) for col in tb_all_fmt.columns], names=['model', 'pooling']).to_frame().T

# Prepend multiheader to tb_all and ignore dtypes
with open(SAVE_PATH, "a") as f:
    f.write("\n\n")
    f.write("# OVERALL RESULTS")
    f.write("\n")
    f.write("mean AUROC over folds per classifier")
    f.write("\n")
# multiheader.to_markdown(SAVE_PATH, mode="a", tablefmt="plain, index=True)
# with open(SAVE_PATH, "a") as f:
#     f.write("\n")
#     f.write("|")
#     for i in range(len(multiheader.columns)):
#         f.write("----|")
#     f.write("\n")
multiheader.to_markdown(SAVE_PATH, mode="a", tablefmt="github", index=True)
tb_all_fmt.to_markdown(SAVE_PATH, mode="a", tablefmt="github", index=True, floatfmt='.3f')


In [110]:

# Calculate the mean AUROC for each classifier
summary_table = tb_all.mean(axis=1).to_frame(name='mean AUROC').T

# Save the summary table to the markdown file
with open(SAVE_PATH, "a") as f:
    f.write("\n\n")
    f.write("# SUMMARY TABLE")
    f.write("\n")
    f.write("Average AUROC over all labels per classifier")
    f.write("\n")

# Display the summary table
summary_table.to_markdown(SAVE_PATH, mode="a", tablefmt="github", index=True, floatfmt='.3f')
